this report generates all plots for ablation study.


there are four sets of data to read

1. original recurrent models, 2L, 16/32 ch, average depth as well as performance.
2. multipath models, 2L, 16/32 ch, average depth as well as performance.
3. ablated multipath models, onlyDX, 2L, 16/32 ch, average depth as well as performance.
4. ablated multipath models, leDX/geDX, 2L, 16/32 ch, average depth as well as performance.

I should denote them with different colors.


I should plot data at two levels.

1. overall level
2. per read out type. this is to show that 8K data does not work well with "single path does not work" hypothesis


storage format.

a big dataframe

* `data_source`: 'original', 'multipath', 'onlyDX'. 'leDXgeDX'
* `aggregate_level`: '', 'readout_type=inst-avg', 'readout_type=inst-last', 'readout_type=cm-avg', 'readout_type=cm-last'
    * '' means everything
    * more general, it's `/` joined by `k=v`
* `rcnn_bl_cls`: 2,3,4,5,6,7
    * no feed-forward models involved
* `n`: how many raw models are involved to get every number that follows
* `depth_scalar_mean`: a list of depths.
    * for `leDX/geDX`, this will be a proper list, with 2*`rcnn_bl_cls` numbers, following the order in plots in `/results_processed/tang_refactored/maskcnn_polished_with_rcnn_k_bl/20201218_plot.ipynb`
    * for others, this will be a list with a single number.
* `depth_scalar_sem`: similar to above, but sem. calculated with `sem(ddof=0)`.
* `cc2_normed_avg_mean`: similar to `depth_scalar_mean`
* `cc2_normed_avg_sem`: similar to `depth_scalar_sem`


Note that, for every combination of (`data_source`, `aggregate_level`, `rcnn_bl_cls`), the `n` should be the same and only and all models that are available in `multipath` should be analzyed. This is because `multipath` models have the most demanding requirement on memory so there are fewest of them.

In [1]:
from os.path import join, dirname, exists
from os import makedirs
from thesis_v2 import dir_dict

In [2]:
import pandas as pd
import numpy as np

In [3]:
from thesis_v2.analysis.ablation_study import collect_all_data

In [4]:
def load_original_perf():
    f_1 = join(dir_dict['analyses'], 'tang_refactored', '20201002', 'main_result_0,500.pkl')
    f_2 = join(dir_dict['analyses'], 'tang_refactored', '20201018', 'main_result_0,500.pkl')
    ret = pd.concat([
        pd.read_pickle(f_1),
        pd.read_pickle(f_2),
    ], axis=0).sort_index()
    
    return ret

In [5]:
def load_original_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'tang_refactored', '20201002_500+20201018', 'source_analysis.pkl')).sort_index()
    return ret

In [6]:
def load_multipath_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'tang_refactored', '20201215', 'main_result_separatebn.pkl')),
        ], axis=0
    )
    return ret

def load_multipath_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'tang_refactored', '20201215', 'source_analysis.pkl'))
    return ret

In [7]:
def load_onlyDX_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'tang_refactored', '20201221', 'main_result.pkl')),
        ], axis=0
    )
    return ret

In [8]:
def load_onlyDX_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'tang_refactored', '20201221', 'source_analysis.pkl'))
    return ret

In [9]:
def load_leDXgeDX_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'tang_refactored', '20201218', 'main_result.pkl')),
        ], axis=0
    )
    return ret

In [10]:
def load_leDXgeDX_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'tang_refactored', '20201218', 'source_analysis.pkl'))
    return ret

In [11]:
f_main_result = join(dir_dict['analyses'], 'tang_refactored', 'ablation_2L_16,32ch_1400.pkl')
makedirs(dirname(f_main_result), exist_ok=True)

if not exists(f_main_result):
    df_main_result = collect_all_data(
    data_loader_dict = {
        'multipath': {
            'perf': load_multipath_perf,
            'depth': load_multipath_depth,
        },
        'original': {
            'perf': load_original_perf,
            'depth': load_original_depth,
        },
        'onlyDX': {
            'perf': load_onlyDX_perf,
            'depth': load_onlyDX_depth,
        },
        'leDXgeDX': {
            'perf': load_leDXgeDX_perf,
            'depth': load_leDXgeDX_depth,
        }
    },
        train_keep=1400,
    )
    df_main_result.to_pickle(f_main_result)
    del df_main_result
df_main_result = pd.read_pickle(f_main_result)

(768,)


In [12]:
df_main_result

depth_mean  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                                                   [0.0, 1.0]   
                                   3            [0.0, 0.7349455866004018, 1.0126414590101174, ...   
                                   4            [0.0, 0.6697849706239946, 1.246276222760816, 1...   
                                   5            [0.0, 0.6661451227797581, 1.0638647085546662, ...   
                                   6            [0.0, 0.6616039534874103, 1.0480180914933765, ...   
...                                                                                           ...   
original    readout_type=inst-last 3                                         [1.6590407206364262]   
                                   4                                          [2.417850031673957]   
                                   5                                         [3.1520510333665275]   
                                   6                                          [3.859972426331696]   
                                   7                                          [4.450262986766662]   

                                                                                        depth_sem  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                                                   [0.0, 0.0]   
                                   3            [0.0, 0.010514020429061084, 0.0008842524479652...   
                                   4            [0.0, 0.011783369663059233, 0.0193850134789104...   
                                   5            [0.0, 0.011428458099071769, 0.0178110038737337...   
                                   6            [0.0, 0.01156864620227423, 0.02235151552471544...   
...                                                                                           ...   
original    readout_type=inst-last 3                                         [0.0130070953010654]   
                                   4                                        [0.03454727950625677]   
                                   5                                       [0.056833613464677156]   
                                   6                                        [0.07979576850924189]   
                                   7                                        [0.11136148483451801]   

                                                  n  \
data_source aggregate_level        rcnn_bl_cls        
leDXgeDX                           2            128   
                                   3            128   
                                   4            128   
                                   5            128   
                                   6            128   
...                                             ...   
original    readout_type=inst-last 3             32   
                                   4             32   
                                   5             32   
                                   6             32   
                                   7             32   

                                                                                        perf_mean  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                    [0.3884324597863596, 0.45858856552181987]   
                                   3            [0.38742772283224125, 0.4596537916607479, 0.46...   
                                   4            [0.3873653873093087, 0.4613191888362087, 0.476...   
                                   5            [0.3879696283472718, 0.4619570421712901, 0.476...   
                                   6            [0.3881492684737344, 0.4622154869578572, 0.478...   
...                                 